# 🌱 FasalVaidya NPK Deficiency Detection Model Training

**Optimized for Google Colab with GPU Acceleration**

This notebook trains an EfficientNetV2-S model to detect NPK (Nitrogen, Phosphorus, Potassium) deficiencies in plant leaves.

## Features:

- ⚡ **GPU Acceleration** - Uses Colab's free GPU (T4/V100)
- 🔄 **Parallel Data Loading** - Multi-threaded data pipelines
- 📊 **9-Class Classification** - NPK + healthy + 5 other deficiencies
- 🎯 **Transfer Learning** - EfficientNetV2-S pretrained on ImageNet
- 📈 **3-Phase Training** - Progressive unfreezing for best accuracy

**Runtime → Change runtime type → GPU (T4)** before running!


In [7]:
# 📦 Step 1: Setup & GPU Check
import os
import sys
import shutil
import random
import warnings
from pathlib import Path
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
import multiprocessing

warnings.filterwarnings('ignore')

# Install and import tqdm for progress bars
%pip install -q tqdm

from tqdm.auto import tqdm

# Check GPU availability
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU devices: {tf.config.list_physical_devices('GPU')}")

# Enable memory growth to prevent OOM
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print(f"✅ GPU enabled: {gpus[0].name}")
else:
    print("⚠️ No GPU found! Go to Runtime → Change runtime type → GPU")

# Set optimal thread counts
NUM_PARALLEL_CALLS = tf.data.AUTOTUNE
NUM_WORKERS = multiprocessing.cpu_count()
print(f"CPU cores for parallel loading: {NUM_WORKERS}")

# Detect environment (Colab vs Local)
IS_COLAB = 'google.colab' in sys.modules
print(f"Running on: {'Google Colab' if IS_COLAB else 'Local Machine'}")

TensorFlow version: 2.19.0
GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
✅ GPU enabled: /physical_device:GPU:0
CPU cores for parallel loading: 2
Running on: Google Colab


In [9]:
# 📁 Step 2: Setup Paths (Auto-detect Colab vs Local)
import os
from pathlib import Path

# Auto-detect environment and set paths accordingly
if IS_COLAB:
    print("📍 Running on Google Colab")
    
    # Try to mount Google Drive
    DRIVE_MOUNTED = False
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
        DRIVE_MOUNTED = True
        print("   ✅ Google Drive mounted successfully")
    except Exception as e:
        print(f"   ⚠️ Google Drive mount failed: {e}")
        print("   📂 Using Colab's local storage instead")
    
    if DRIVE_MOUNTED:
        # Use Google Drive for persistent storage
        BASE_DIR = Path("/content/drive/MyDrive/FasalVaidya/ml/models")
        # Create directory if it doesn't exist
        BASE_DIR.mkdir(parents=True, exist_ok=True)
    else:
        # Fallback: Use Colab's local storage (faster but not persistent)
        BASE_DIR = Path("/content/ml_models")
        BASE_DIR.mkdir(parents=True, exist_ok=True)
        print("\n   ⚠️ IMPORTANT: Local storage is NOT persistent!")
        print("   📤 Upload your datasets to /content/ml_models/ using the file browser")
        print("   💾 Download your trained model before session ends!")
else:
    # Local Windows paths
    BASE_DIR = Path(r"E:\FasalVaidya\backend\ml\models")
    print("📍 Running on Local Machine")

# Dataset paths - all your datasets
# NOTE: "Bigger CoLeaf DATASET" has nested "CoLeaf DATASET" folder with actual classes
DATA_SOURCES = [
    BASE_DIR / "Bigger CoLeaf DATASET" / "CoLeaf DATASET",  # Main 9-class dataset
    BASE_DIR / "Propossed_Data" / "Contrast_Stretching",
    BASE_DIR / "Propossed_Data" / "Histogram_Equalization",
    BASE_DIR / "Propossed_Data" / "Log_Transformation",
    # Additional datasets
    BASE_DIR / "Nitrogen deficiency",  # Rice nitrogen dataset (train/val/test → control/deficiency)
    BASE_DIR / "ThorCam_semiFiltered",  # Control (-C) & Phosphorus (-P, -P50)
    BASE_DIR / "POTASSIUM DEFICIENCY",  # Extra potassium data (flat folder, all K deficiency)
]

# Output paths
OUTPUT_DIR = BASE_DIR
OUTPUT_MODEL = OUTPUT_DIR / "plantvillage-npk-v3.h5"

# Local cache for combined dataset
LOCAL_CACHE = BASE_DIR / "combined_balanced_dataset"
LOCAL_CACHE.mkdir(parents=True, exist_ok=True)

print(f"\n📁 Base directory: {BASE_DIR}")
print(f"💾 Cache directory: {LOCAL_CACHE}")
print(f"📦 Output model: {OUTPUT_MODEL}")

# Verify paths exist
print("\n🔍 Checking dataset paths:")
found_any = False
for src in DATA_SOURCES:
    if src.exists():
        found_any = True
        count = sum(1 for _ in src.rglob("*") if _.is_file() and _.suffix.lower() in ['.jpg', '.jpeg', '.png', '.tif', '.tiff'])
        print(f"   ✅ {src.name}: ~{count} images")
    else:
        print(f"   ⚠️ NOT FOUND: {src}")

if not found_any and IS_COLAB:
    print("\n" + "=" * 60)
    print("📤 NO DATASETS FOUND! Upload your data:")
    print("=" * 60)
    print(f"   1. Click the folder icon (📁) in the left sidebar")
    print(f"   2. Navigate to: {BASE_DIR}")
    print(f"   3. Upload your dataset folders:")
    print(f"      - Bigger CoLeaf DATASET/")
    print(f"      - Propossed_Data/")
    print(f"      - etc.")
    print("=" * 60)

📍 Running on Google Colab
   ⚠️ Google Drive mount failed: mount failed
   📂 Using Colab's local storage instead

   ⚠️ IMPORTANT: Local storage is NOT persistent!
   📤 Upload your datasets to /content/ml_models/ using the file browser
   💾 Download your trained model before session ends!

📁 Base directory: /content/ml_models
💾 Cache directory: /content/ml_models/combined_balanced_dataset
📦 Output model: /content/ml_models/plantvillage-npk-v3.h5

🔍 Checking dataset paths:
   ⚠️ NOT FOUND: /content/ml_models/Bigger CoLeaf DATASET/CoLeaf DATASET
   ⚠️ NOT FOUND: /content/ml_models/Propossed_Data/Contrast_Stretching
   ⚠️ NOT FOUND: /content/ml_models/Propossed_Data/Histogram_Equalization
   ⚠️ NOT FOUND: /content/ml_models/Propossed_Data/Log_Transformation
   ⚠️ NOT FOUND: /content/ml_models/Nitrogen deficiency
   ⚠️ NOT FOUND: /content/ml_models/ThorCam_semiFiltered
   ⚠️ NOT FOUND: /content/ml_models/POTASSIUM DEFICIENCY

📤 NO DATASETS FOUND! Upload your data:
   1. Click the folder ic

In [10]:
# 📤 Step 2b: Upload Dataset to Colab
# Run this cell DIRECTLY in Colab browser (not via VS Code remote connection)

if IS_COLAB:
    print("=" * 70)
    print("📤 DATASET UPLOAD OPTIONS")
    print("=" * 70)
    
    # ═══════════════════════════════════════════════════════════════════════
    # OPTION 1: Manual Upload via File Browser (RECOMMENDED)
    # ═══════════════════════════════════════════════════════════════════════
    print("""
🔹 OPTION 1: Manual Upload (Works Always)
   1. Click the folder icon (📁) in the LEFT sidebar of Colab
   2. Right-click on the file browser → 'Upload'
   3. Upload your ZIP file containing the datasets
   4. Then run the extraction code below
    """)
    
    # Check if any zip files were uploaded manually
    import glob 
    zip_files = glob.glob("/content/*.zip")
    if zip_files:
        print(f"   ✅ Found uploaded ZIP files: {zip_files}")
        print("   Run the extraction cell below!")
    else:
        print("   ⏳ No ZIP files found yet. Upload one first.")
    
    print("""
═══════════════════════════════════════════════════════════════════════
🔹 OPTION 2: Download from Google Drive Link
   If you have a shareable Google Drive link, uncomment and run:
═══════════════════════════════════════════════════════════════════════
    """)

# ═══════════════════════════════════════════════════════════════════════════
# OPTION 2: Download from Google Drive shareable link (UNCOMMENT TO USE)
# Replace YOUR_FILE_ID with the actual file ID from your Drive link
# ═══════════════════════════════════════════════════════════════════════════
# !pip install -q gdown
# !gdown "YOUR_FILE_ID" -O /content/dataset.zip
# # Example: !gdown "1ABC123xyz" -O /content/dataset.zip
# # Get FILE_ID from link like: https://drive.google.com/file/d/FILE_ID/view

📤 Upload your dataset as a ZIP file...
   The ZIP should contain folders like: Bigger CoLeaf DATASET/, Propossed_Data/, etc.

⏳ Waiting for file upload (click 'Choose Files' below)...



KeyboardInterrupt: 

In [ ]:
# 📦 Step 2c: Extract uploaded ZIP file
# Run this AFTER uploading a ZIP file to /content/

import zipfile
import glob

zip_files = glob.glob("/content/*.zip")

if zip_files:
    for zip_path in zip_files:
        print(f"📦 Extracting {zip_path}...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(str(BASE_DIR))
        print(f"   ✅ Extracted to {BASE_DIR}")
        
        # Optionally remove zip to save space
        # os.remove(zip_path)
        # print(f"   🗑️ Removed {zip_path}")
    
    # Show what was extracted
    print("\n🔍 Extracted contents:")
    for item in sorted(BASE_DIR.iterdir()):
        if item.is_dir():
            count = sum(1 for _ in item.rglob("*") if _.is_file())
            print(f"   📂 {item.name}/  ({count} files)")
        else:
            print(f"   📄 {item.name}")
else:
    print("⚠️ No ZIP files found in /content/")
    print("   Upload a ZIP file using the file browser (📁) first!")

In [ ]:
# ⚙️ Step 3: Configuration & Auto-detect Dataset Paths
import numpy as np
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import EfficientNetV2S, EfficientNetB0
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict

# Classes - mapping for different folder naming conventions
CLASS_MAPPING = {
    "healthy": "healthy", "control": "healthy", "-C": "healthy",
    "nitrogen-N": "nitrogen-N", "deficiency": "nitrogen-N", "N": "nitrogen-N",
    "phosphorus-P": "phosphorus-P", "-P": "phosphorus-P", "-P50": "phosphorus-P", "P": "phosphorus-P",
    "potasium-K": "potasium-K", "K": "potasium-K",
    "boron-B": "boron-B", "calcium-Ca": "calcium-Ca", "iron-Fe": "iron-Fe",
    "magnesium-Mg": "magnesium-Mg", "manganese-Mn": "manganese-Mn",
}

TARGET_CLASSES = ["healthy", "nitrogen-N", "phosphorus-P", "potasium-K"]
EXTRA_CLASSES = ["boron-B", "calcium-Ca", "iron-Fe", "magnesium-Mg", "manganese-Mn"]
ALL_CLASSES = TARGET_CLASSES + EXTRA_CLASSES

# Training hyperparameters - OPTIMIZED FOR LOCAL CPU/GPU
IMG_SIZE = 224
BATCH_SIZE = 16  # Smaller for CPU
EPOCHS_PHASE1 = 30  # Head training
EPOCHS_PHASE2 = 25  # Fine-tuning top 40%
EPOCHS_PHASE3 = 20  # Full fine-tuning
VALIDATION_SPLIT = 0.15
TEST_SPLIT = 0.10

# Additional hyperparameters
INITIAL_LR = 1e-3
FINE_TUNE_LR = 1e-4
DEEP_TUNE_LR = 1e-5
LABEL_SMOOTHING = 0.1
DROPOUT_RATE = 0.4  # Increased for better regularization
EARLY_STOPPING_PATIENCE = 8

# Class balancing
TARGET_SAMPLES_PER_CLASS = 500  # Oversample/undersample to this target

# Performance settings
PREFETCH_BUFFER = tf.data.AUTOTUNE
SHUFFLE_BUFFER = 2000
NUM_WORKERS = min(8, os.cpu_count() or 4)

print(f"✅ Configuration loaded")
print(f"   Image size: {IMG_SIZE}x{IMG_SIZE}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Classes: {len(ALL_CLASSES)}")
print(f"   Target samples/class: {TARGET_SAMPLES_PER_CLASS}")
print(f"   Training phases: {EPOCHS_PHASE1} + {EPOCHS_PHASE2} + {EPOCHS_PHASE3} = {EPOCHS_PHASE1+EPOCHS_PHASE2+EPOCHS_PHASE3} epochs")

In [ ]:
# 📂 Step 4: Parallel Dataset Combining (Multi-threaded with tqdm)
import shutil
import random
from concurrent.futures import ThreadPoolExecutor, as_completed

def copy_file(args):
    """Thread worker for parallel file copying."""
    src, dest = args
    try:
        if not dest.exists():
            shutil.copy2(src, dest)
            return 1
    except Exception as e:
        pass
    return 0

def collect_all_images():
    """Scan all datasets and collect image paths by class."""
    print("📁 Scanning all datasets...")
    
    class_images = {cls: [] for cls in ALL_CLASSES}
    
    for dataset_path in DATA_SOURCES:
        if not dataset_path.exists():
            print(f"   ⚠️ Not found: {dataset_path}")
            continue
        
        dataset_name = dataset_path.name
        print(f"   📂 Scanning {dataset_name}...")
        
        # Handle Nitrogen deficiency dataset (special structure: train/val/test → control/deficiency)
        if "Nitrogen" in dataset_name or "nitrogen" in dataset_name:
            for split in ["train", "val", "test"]:
                split_dir = dataset_path / split
                if split_dir.exists():
                    for class_dir in split_dir.iterdir():
                        if class_dir.is_dir():
                            std_class = CLASS_MAPPING.get(class_dir.name)
                            if std_class and std_class in class_images:
                                for img in class_dir.glob("*"):
                                    if img.suffix.lower() in [".jpg", ".jpeg", ".png", ".bmp"]:
                                        class_images[std_class].append(img)
        
        # Handle ThorCam dataset (-C, -P, -P50 folders)
        elif "ThorCam" in dataset_name:
            for class_dir in dataset_path.iterdir():
                if class_dir.is_dir():
                    std_class = CLASS_MAPPING.get(class_dir.name)
                    if std_class and std_class in class_images:
                        for img in class_dir.glob("*"):
                            if img.suffix.lower() in [".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".tif"]:
                                class_images[std_class].append(img)
        
        # Handle POTASSIUM DEFICIENCY dataset (flat folder - all images are potassium deficiency)
        elif "POTASSIUM" in dataset_name.upper():
            for img in dataset_path.glob("*"):
                if img.suffix.lower() in [".jpg", ".jpeg", ".png", ".bmp"]:
                    class_images["potasium-K"].append(img)
        
        # Standard structure: class_name/images
        else:
            for class_dir in dataset_path.iterdir():
                if class_dir.is_dir():
                    std_class = CLASS_MAPPING.get(class_dir.name, class_dir.name)
                    if std_class and std_class in class_images:
                        for img in class_dir.glob("*"):
                            if img.suffix.lower() in [".jpg", ".jpeg", ".png", ".bmp"]:
                                class_images[std_class].append(img)
    
    return class_images

def balance_classes(class_images):
    """Balance classes with oversampling/undersampling."""
    print(f"\n⚖️ Balancing classes (target: {TARGET_SAMPLES_PER_CLASS} per class)...")
    
    balanced = {cls: list(imgs) for cls, imgs in class_images.items()}
    
    for cls in ALL_CLASSES:
        current = len(balanced[cls])
        if current == 0:
            print(f"   ⚠️ {cls}: No samples!")
            continue
        
        if current < TARGET_SAMPLES_PER_CLASS:
            # Oversample
            needed = TARGET_SAMPLES_PER_CLASS - current
            balanced[cls].extend(random.choices(balanced[cls], k=needed))
            print(f"   📈 {cls}: {current} → {len(balanced[cls])} (+{needed})")
        elif current > TARGET_SAMPLES_PER_CLASS * 2:
            # Undersample only if way too many
            balanced[cls] = random.sample(balanced[cls], TARGET_SAMPLES_PER_CLASS)
            print(f"   📉 {cls}: {current} → {len(balanced[cls])}")
        else:
            print(f"   ✓ {cls}: {current}")
    
    return balanced

def combine_datasets_parallel():
    """Combine all datasets using parallel threads."""
    # Remove old cache
    if LOCAL_CACHE.exists():
        print("   Removing old cache...")
        shutil.rmtree(LOCAL_CACHE)
    
    # Collect all images
    class_images = collect_all_images()
    
    # Print raw stats
    print("\n📊 Raw dataset statistics:")
    total_raw = 0
    max_count = max(len(imgs) for imgs in class_images.values()) if class_images else 1
    for cls in ALL_CLASSES:
        count = len(class_images[cls])
        total_raw += count
        bar = "█" * int(30 * count / max_count) if max_count > 0 else ""
        print(f"   {cls:15s}: {bar:30s} {count:5d}")
    print(f"   {'─' * 50}")
    print(f"   {'TOTAL':15s}:                                {total_raw:5d}")
    
    # Balance classes
    balanced_images = balance_classes(class_images)
    
    # Create directories
    for cls in ALL_CLASSES:
        (LOCAL_CACHE / cls).mkdir(parents=True, exist_ok=True)
    
    # Prepare copy tasks
    copy_tasks = []
    for cls in ALL_CLASSES:
        for i, src_path in enumerate(balanced_images[cls]):
            suffix = src_path.suffix.lower()
            if suffix not in [".jpg", ".jpeg", ".png"]:
                suffix = ".jpg"
            dst_path = LOCAL_CACHE / cls / f"{cls}_{i:05d}{suffix}"
            copy_tasks.append((src_path, dst_path))
    
    # Copy with progress bar
    print(f"\n📁 Copying {len(copy_tasks)} files with {NUM_WORKERS} threads...")
    copied = 0
    with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
        futures = list(tqdm(
            executor.map(copy_file, copy_tasks),
            total=len(copy_tasks),
            desc="   Copying",
            unit="file"
        ))
        copied = sum(futures)
    
    print(f"   ✅ Done! {copied} files copied to {LOCAL_CACHE}")
    return balanced_images

# Run the combination
balanced_data = combine_datasets_parallel()

In [ ]:
# 🔀 Step 5: Create Train/Val/Test Splits
def create_splits():
    """Create stratified train/val/test splits."""
    print("\n🔀 Creating train/val/test splits...")
    
    all_images = []
    all_labels = []
    class_to_idx = {cls: idx for idx, cls in enumerate(ALL_CLASSES)}
    
    for cls in ALL_CLASSES:
        cls_dir = LOCAL_CACHE / cls
        if not cls_dir.exists():
            continue
        for img_file in cls_dir.glob("*"):
            if img_file.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp']:
                all_images.append(str(img_file))
                all_labels.append(class_to_idx[cls])
    
    all_images = np.array(all_images)
    all_labels = np.array(all_labels)
    
    # Stratified splits
    train_val_imgs, test_imgs, train_val_labels, test_labels = train_test_split(
        all_images, all_labels, test_size=TEST_SPLIT, stratify=all_labels, random_state=42
    )
    
    val_ratio = VALIDATION_SPLIT / (1 - TEST_SPLIT)
    train_imgs, val_imgs, train_labels, val_labels = train_test_split(
        train_val_imgs, train_val_labels, test_size=val_ratio, stratify=train_val_labels, random_state=42
    )
    
    print(f"   ✅ Train: {len(train_imgs):4d} images")
    print(f"   ✅ Val:   {len(val_imgs):4d} images")
    print(f"   ✅ Test:  {len(test_imgs):4d} images")
    
    return (train_imgs, train_labels), (val_imgs, val_labels), (test_imgs, test_labels)

(train_imgs, train_labels), (val_imgs, val_labels), (test_imgs, test_labels) = create_splits()
NUM_CLASSES = len(ALL_CLASSES)
print(f"\n📚 Classes ({NUM_CLASSES}): {ALL_CLASSES}")

In [ ]:
# ⚡ Step 6: Optimized tf.data Pipeline (GPU-accelerated)
AUTOTUNE = tf.data.AUTOTUNE

def augment(image, training=True):
    """GPU-accelerated augmentation."""
    if training:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        image = tf.image.random_brightness(image, max_delta=0.2)
        image = tf.image.random_contrast(image, 0.8, 1.2)
        image = tf.image.random_saturation(image, 0.8, 1.2)
        image = tf.image.random_hue(image, max_delta=0.1)
        # Random rotation (90° increments)
        k = tf.random.uniform([], 0, 4, dtype=tf.int32)
        image = tf.image.rot90(image, k)
    return image

def parse_image(file_path, label, training=True):
    """Load and preprocess image with parallel threading."""
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32) / 255.0
    if training:
        image = augment(image, training=True)
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

def create_dataset(images, labels, training=True):
    """Create optimized tf.data.Dataset with parallelization."""
    ds = tf.data.Dataset.from_tensor_slices((images, labels))
    if training:
        ds = ds.shuffle(buffer_size=len(images), reshuffle_each_iteration=True)
    
    # Parallel map with NUM_WORKERS threads
    ds = ds.map(
        lambda x, y: parse_image(x, y, training),
        num_parallel_calls=AUTOTUNE
    )
    
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(AUTOTUNE)
    
    if training:
        ds = ds.cache()  # Cache after batching for memory efficiency
    
    return ds

# Create datasets
print("⚡ Creating optimized GPU data pipelines...")
train_ds = create_dataset(train_imgs, train_labels, training=True)
val_ds = create_dataset(val_imgs, val_labels, training=False)
test_ds = create_dataset(test_imgs, test_labels, training=False)

# Calculate steps
steps_per_epoch = len(train_imgs) // BATCH_SIZE
validation_steps = len(val_imgs) // BATCH_SIZE

print(f"   ✅ Train batches: {len(train_imgs) // BATCH_SIZE}")
print(f"   ✅ Val batches:   {len(val_imgs) // BATCH_SIZE}")
print(f"   ✅ Prefetching:   AUTOTUNE enabled")

In [ ]:
# 🏗️ Step 7: Build EfficientNetV2-S Model
def build_model():
    """Build EfficientNetV2-S with custom head for NPK classification."""
    print("\n🏗️ Building EfficientNetV2-S model...")
    
    # Base model (pretrained on ImageNet)
    base_model = tf.keras.applications.EfficientNetV2S(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights='imagenet',
        pooling=None
    )
    
    # Initially freeze all layers
    base_model.trainable = False
    
    # Build model with custom head
    inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(DROPOUT_RATE)(x)
    x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(DROPOUT_RATE)(x)
    x = tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = tf.keras.layers.Dropout(DROPOUT_RATE / 2)(x)
    outputs = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)
    
    model = tf.keras.Model(inputs, outputs)
    
    print(f"   ✅ Base model:     EfficientNetV2-S")
    print(f"   ✅ Total params:   {model.count_params():,}")
    print(f"   ✅ Trainable:      {sum(tf.keras.backend.count_params(w) for w in model.trainable_weights):,}")
    
    return model, base_model

model, base_model = build_model()
model.summary()

In [ ]:
# ⚖️ Step 8: Calculate Class Weights (Handle Imbalance)
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weight_dict = dict(enumerate(class_weights))

print("⚖️ Class weights for imbalanced data:")
for cls_name, weight in zip(ALL_CLASSES, class_weights):
    print(f"   {cls_name:15s}: {weight:.3f}")

In [ ]:
# 🎯 Step 9: Training Callbacks with tqdm Progress Bar
import datetime

# Create output directories
CHECKPOINT_DIR = OUTPUT_DIR / "checkpoints"
LOGS_DIR = OUTPUT_DIR / "logs"
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)
LOGS_DIR.mkdir(parents=True, exist_ok=True)

# Custom tqdm callback for training progress
class TqdmCallback(tf.keras.callbacks.Callback):
    def __init__(self, epochs, phase_name="Training"):
        super().__init__()
        self.epochs = epochs
        self.phase_name = phase_name
        self.epoch_bar = None
        
    def on_train_begin(self, logs=None):
        self.epoch_bar = tqdm(total=self.epochs, desc=f"🚀 {self.phase_name}", 
                              unit="epoch", position=0, leave=True,
                              bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]')
    
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        # Update progress bar with metrics
        self.epoch_bar.set_postfix({
            'loss': f"{logs.get('loss', 0):.4f}",
            'acc': f"{logs.get('accuracy', 0):.2%}",
            'val_acc': f"{logs.get('val_accuracy', 0):.2%}"
        })
        self.epoch_bar.update(1)
    
    def on_train_end(self, logs=None):
        self.epoch_bar.close()

# Training callbacks
callbacks_base = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=EARLY_STOPPING_PATIENCE,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        str(CHECKPOINT_DIR / "best_model.keras"),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=0  # Reduced verbosity for cleaner output
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir=str(LOGS_DIR / datetime.datetime.now().strftime("%Y%m%d-%H%M%S")),
        histogram_freq=1
    )
]

print("🎯 Training callbacks configured:")
print("   ✅ TqdmCallback (progress bar)")
print("   ✅ EarlyStopping (patience={})".format(EARLY_STOPPING_PATIENCE))
print("   ✅ ReduceLROnPlateau")
print("   ✅ ModelCheckpoint")
print("   ✅ TensorBoard")

In [ ]:
# 🚀 Step 10: Phase 1 - Train Head Only (Frozen Backbone)
print("=" * 60)
print("🚀 PHASE 1: Training Classification Head (Backbone Frozen)")
print("=" * 60)

# Compile model for Phase 1
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=INITIAL_LR),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=LABEL_SMOOTHING),
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top3_accuracy')]
)

# Create Phase 1 callbacks with tqdm
phase1_callbacks = callbacks_base + [TqdmCallback(EPOCHS_PHASE1, "Phase 1: Head")]

# Train Phase 1
history_phase1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_PHASE1,
    callbacks=phase1_callbacks,
    class_weight=class_weight_dict,
    verbose=0  # Suppress default output, use tqdm instead
)

print("\n✅ Phase 1 Complete!")
print(f"   Best val_accuracy: {max(history_phase1.history['val_accuracy']):.4f}")

In [ ]:
# 🔥 Step 11: Phase 2 - Unfreeze Top Layers (Progressive Unfreezing)
print("\n" + "=" * 60)
print("🔥 PHASE 2: Training Top 30% of Backbone")
print("=" * 60)

# Unfreeze top 30% of base model layers
base_model.trainable = True
num_layers = len(base_model.layers)
freeze_until = int(num_layers * 0.7)  # Freeze first 70%

for layer in base_model.layers[:freeze_until]:
    layer.trainable = False

trainable_count = sum(1 for layer in base_model.layers if layer.trainable)
print(f"   📊 Total layers: {num_layers}")
print(f"   🔒 Frozen layers: {freeze_until}")
print(f"   🔥 Trainable layers: {trainable_count}")

# Recompile with lower LR
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=FINE_TUNE_LR),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=LABEL_SMOOTHING),
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top3_accuracy')]
)

# Create Phase 2 callbacks with tqdm
phase2_callbacks = callbacks_base + [TqdmCallback(EPOCHS_PHASE2, "Phase 2: Top 30%")]

# Train Phase 2
history_phase2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_PHASE2,
    callbacks=phase2_callbacks,
    class_weight=class_weight_dict,
    verbose=0  # Suppress default output, use tqdm instead
)

print("\n✅ Phase 2 Complete!")
print(f"   Best val_accuracy: {max(history_phase2.history['val_accuracy']):.4f}")

In [ ]:
# 🎓 Step 12: Phase 3 - Full Fine-tuning (All Layers)
print("\n" + "=" * 60)
print("🎓 PHASE 3: Full Fine-tuning (All Layers Trainable)")
print("=" * 60)

# Unfreeze entire backbone
base_model.trainable = True
for layer in base_model.layers:
    layer.trainable = True

print(f"   🔥 All {len(base_model.layers)} layers now trainable")
print(f"   📊 Total trainable params: {sum(tf.keras.backend.count_params(w) for w in model.trainable_weights):,}")

# Recompile with very low LR
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=FINE_TUNE_LR / 5),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=LABEL_SMOOTHING),
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top3_accuracy')]
)

# Create Phase 3 callbacks with tqdm
phase3_callbacks = callbacks_base + [TqdmCallback(EPOCHS_PHASE3, "Phase 3: Full")]

# Train Phase 3
history_phase3 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_PHASE3,
    callbacks=phase3_callbacks,
    class_weight=class_weight_dict,
    verbose=0  # Suppress default output, use tqdm instead
)

print("\n✅ Phase 3 Complete!")
print(f"   Best val_accuracy: {max(history_phase3.history['val_accuracy']):.4f}")

In [ ]:
# 📊 Step 13: Training Visualization
import matplotlib.pyplot as plt

def plot_training_history(histories, phase_names):
    """Plot training curves for all phases."""
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    colors = ['#2ecc71', '#3498db', '#9b59b6']
    
    epoch_offset = 0
    for hist, name, color in zip(histories, phase_names, colors):
        epochs = range(epoch_offset, epoch_offset + len(hist.history['loss']))
        
        # Loss
        axes[0, 0].plot(epochs, hist.history['loss'], color=color, linestyle='-', label=f'{name} Train')
        axes[0, 0].plot(epochs, hist.history['val_loss'], color=color, linestyle='--', label=f'{name} Val')
        
        # Accuracy
        axes[0, 1].plot(epochs, hist.history['accuracy'], color=color, linestyle='-', label=f'{name} Train')
        axes[0, 1].plot(epochs, hist.history['val_accuracy'], color=color, linestyle='--', label=f'{name} Val')
        
        # Top-3 Accuracy
        axes[1, 0].plot(epochs, hist.history['top3_accuracy'], color=color, linestyle='-', label=f'{name} Train')
        axes[1, 0].plot(epochs, hist.history['val_top3_accuracy'], color=color, linestyle='--', label=f'{name} Val')
        
        epoch_offset += len(hist.history['loss'])
    
    axes[0, 0].set_title('Loss', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].legend(fontsize=8)
    axes[0, 0].grid(True, alpha=0.3)
    
    axes[0, 1].set_title('Accuracy', fontsize=12, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].legend(fontsize=8)
    axes[0, 1].grid(True, alpha=0.3)
    
    axes[1, 0].set_title('Top-3 Accuracy', fontsize=12, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].legend(fontsize=8)
    axes[1, 0].grid(True, alpha=0.3)
    
    # Learning rate visualization
    axes[1, 1].text(0.5, 0.5, f"🏆 Training Summary\n\nPhase 1: Head Only\nPhase 2: Top 30% Backbone\nPhase 3: Full Fine-tuning\n\nBest Val Accuracy:\n{max(history_phase3.history['val_accuracy']):.2%}", 
                    ha='center', va='center', fontsize=12, transform=axes[1, 1].transAxes)
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.savefig(str(OUTPUT_DIR / 'training_history.png'), dpi=150)
    plt.show()

plot_training_history(
    [history_phase1, history_phase2, history_phase3],
    ['Phase 1', 'Phase 2', 'Phase 3']
)

In [ ]:
# 🧪 Step 14: Model Evaluation on Test Set
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

print("=" * 60)
print("🧪 FINAL EVALUATION ON TEST SET")
print("=" * 60)

# Evaluate on test set with progress bar
print("\n📊 Evaluating model...")
test_loss, test_acc, test_top3 = model.evaluate(test_ds, verbose=1)
print(f"\n📊 Test Results:")
print(f"   Loss:          {test_loss:.4f}")
print(f"   Accuracy:      {test_acc:.2%}")
print(f"   Top-3 Accuracy: {test_top3:.2%}")

# Get predictions with tqdm progress bar
y_true = []
y_pred = []

print("\n🔮 Generating predictions...")
for images, labels in tqdm(test_ds, desc="🔮 Predicting", unit="batch"):
    predictions = model.predict(images, verbose=0)
    y_true.extend(np.argmax(labels.numpy(), axis=1))
    y_pred.extend(np.argmax(predictions, axis=1))

# Classification report
print("\n📋 Classification Report:")
print(classification_report(y_true, y_pred, target_names=ALL_CLASSES, digits=3))

# Confusion Matrix
plt.figure(figsize=(12, 10))
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=ALL_CLASSES, yticklabels=ALL_CLASSES)
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(str(OUTPUT_DIR / 'confusion_matrix.png'), dpi=150)
plt.show()

In [ ]:
# 💾 Step 15: Save Final Model
print("=" * 60)
print("💾 SAVING FINAL MODEL")
print("=" * 60)

# Save in multiple formats
final_model_path = OUTPUT_DIR / "fasalvaidya-npk-efficientnetv2s.h5"
model.save(str(final_model_path))
print(f"   ✅ Saved H5:     {final_model_path}")

# Save as SavedModel format (for TensorFlow Serving)
saved_model_path = OUTPUT_DIR / "saved_model"
model.save(str(saved_model_path))
print(f"   ✅ Saved TF:     {saved_model_path}")

# Save class names
class_names_path = OUTPUT_DIR / "class_names.txt"
with open(class_names_path, 'w') as f:
    for cls in ALL_CLASSES:
        f.write(cls + '\n')
print(f"   ✅ Saved Classes: {class_names_path}")

# Save training config
config = {
    'model': 'EfficientNetV2-S',
    'img_size': IMG_SIZE,
    'num_classes': NUM_CLASSES,
    'classes': ALL_CLASSES,
    'test_accuracy': float(test_acc),
    'test_top3_accuracy': float(test_top3),
    'training_phases': ['head_only', 'top_30_unfrozen', 'full_finetune']
}

import json
config_path = OUTPUT_DIR / "model_config.json"
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)
print(f"   ✅ Saved Config: {config_path}")

print("\n🎉 Model training complete!")

In [ ]:
# 📥 Step 16: Copy Model to Backend
import shutil

print("📥 Copying trained model to backend directory...")

# Set backend path based on environment
if IS_COLAB:
    # On Colab, save to Google Drive
    backend_model_path = Path("/content/drive/MyDrive/FasalVaidya/ml/models/plantvillage-npk-v3.h5")
    print("   ℹ️ On Colab: Model saved to Google Drive")
    print("   📥 Download it and copy to your local backend/ml/models/ folder")
else:
    # Local Windows path
    backend_model_path = Path(r"E:\FasalVaidya\backend\ml\models\plantvillage-npk-v3.h5")

shutil.copy2(final_model_path, backend_model_path)
print(f"   ✅ Copied to: {backend_model_path}")

# Also save a backup
backup_path = OUTPUT_DIR / "plantvillage-npk-v3.h5"
if backup_path != final_model_path:
    shutil.copy2(final_model_path, backup_path)
    print(f"   ✅ Backup at: {backup_path}")

print("\n✅ Model is ready! To use it:")
print(f"   1. Update MODEL_PATH in tasks.json to: {backend_model_path}")
print(f"   2. Restart the backend server")
print(f"   3. Test with new scans!")

---
## 📋 Training Instructions

### 🌐 Google Colab Setup
1. **Upload datasets to Google Drive:**
   ```
   /MyDrive/FasalVaidya/ml/models/
   ├── Bigger CoLeaf DATASET/
   ├── Propossed_Data/
   ├── Nitrogen deficiency/
   ├── ThorCam_semiFiltered/
   └── POTASSIUM DEFICIENCY/
   ```
2. **Run Step 1** - Installs dependencies and detects GPU
3. **Run Step 2** - Mounts Google Drive automatically
4. **Run remaining cells** in order

### 💻 Local Training Setup
Your datasets should be at:
```
E:\FasalVaidya\backend\ml\models\
├── Bigger CoLeaf DATASET/          # Main dataset (9 classes)
├── Propossed_Data/                  # 3 augmentation variants
│   ├── Contrast_Stretching/
│   ├── Histogram_Equalization/
│   └── Log_Transformation/
├── Nitrogen deficiency/             # Extra nitrogen data
├── ThorCam_semiFiltered/            # Phosphorus data
└── POTASSIUM DEFICIENCY/            # Extra potassium data
```

### 2️⃣ Run All Cells in Order
Run each cell sequentially (Shift+Enter):
1. Step 1: Imports & Environment Detection
2. Step 2: Paths (auto-detects Colab vs Local)
3. Step 3: Configuration
4. Step 4: Combine datasets (creates balanced cache)
5. Step 5: Train/Val/Test splits
6. Step 6: Create data pipeline
7. Step 7: Build model
8. Step 8: Class weights
9. Step 9: Callbacks
10. Steps 10-12: Training phases
11. Steps 13-16: Evaluation and save

### 3️⃣ Training Time Estimates
| Environment | Phase 1 | Phase 2 | Phase 3 | Total |
|-------------|---------|---------|---------|-------|
| Colab GPU (T4) | ~15 min | ~20 min | ~15 min | ~50 min |
| Local CPU | ~45 min | ~60 min | ~75 min | ~3 hours |
| Local GPU | ~20 min | ~30 min | ~25 min | ~1.25 hours |

### 4️⃣ After Training
- Model saved as `plantvillage-npk-v3.h5`
- If on Colab: Download from Google Drive
- Update backend to use the new model
- Test with the frontend app

---

**💡 Tips:**

- On Colab: Use GPU runtime (Runtime → Change runtime type → GPU)
- Use smaller batch size (8-16) for CPU training
- Monitor memory usage during training
- Early stopping will kick in if validation loss plateaus
